# Notebook 2: Data Processing & Vector Database

**Objectives:**
- Clean and structure event data
- Use LLM to extract additional metadata (baby-friendliness only)
- Generate embeddings for semantic search
- Set up Qdrant and index all events

**✅✅✅ Important Notes:**
- We're keeping it simple: only extracting `baby_friendly: bool` from LLM
- Semantic search will naturally handle mood/vibe queries
- Event-level chunks (1 event = 1 document)

---


## Setup & Imports


In [ ]:
# Install required packages if needed
# !pip install openai qdrant-client pandas numpy python-dotenv


In [1]:
import pandas as pd
import numpy as np
import os
import json
from datetime import datetime
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import time
from tqdm import tqdm

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"), organization=os.getenv("OPENAI_ORG_ID"))

print("✅ Imports successful!")
print(f"OpenAI API Key loaded: {'Yes' if os.getenv('OPENAI_API_KEY') else 'No'}")


✅ Imports successful!
OpenAI API Key loaded: Yes


## 1. Load Raw Data

Load the CSV file from Notebook 1.


In [2]:
os.getenv("OPENAI_ORG_ID")

'org-U43gtMbXKtifsSUvWmXcbydP'

In [3]:
# Find the most recent CSV file in data/raw/
raw_data_dir = Path("../data/raw")
csv_files = list(raw_data_dir.glob("timeout_events_*.csv"))

if not csv_files:
    raise FileNotFoundError("No CSV files found in data/raw/. Please run Notebook 1 first.")

# Get the most recent file
latest_csv = sorted(csv_files)[-1]
print(f"Loading data from: {latest_csv}")

# Load data
df = pd.read_csv(latest_csv)

print(f"\n✅ Loaded {len(df)} events")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst event:")
df.head(1)


Loading data from: ../data/raw/timeout_events_20251019.csv

✅ Loaded 99 events

Columns: ['event_id', 'title', 'description', 'url']

First event:


,event_id,title,description,url
0,evt_001,1.Open House New York,"Admit it, are you a nosy New Yorker? Same here...",https://www.timeout.com/newyork/news/open-hous...


## 2. LLM-Based Feature Extraction

**✅✅✅ Simplified Approach:**
- Extract only `baby_friendly: bool` from event descriptions
- If True, implies stroller-accessible and suitable for infants/toddlers
- No mood tags or energy levels needed - semantic search handles this naturally!

**Why this works:**
- Baby-friendliness is objective and actionable
- Semantic embeddings capture nuanced vibes ("romantic", "relaxing", etc.) automatically
- Reduces API calls and processing time by 50%+


In [4]:
def extract_baby_friendly(title: str, description: str) -> bool:
    """
    Use GPT-4 to determine if an event is baby-friendly.
    
    Args:
        title: Event title
        description: Event description
    
    Returns:
        bool: True if baby-friendly, False otherwise
    """
    prompt = f"""Given this NYC event, determine if it is baby-friendly.

Baby-friendly means:
- Suitable for infants and toddlers (ages 0-3)
- Stroller-accessible
- Family-oriented or welcoming to young children
- Not explicitly adult-only (bars, nightclubs, R-rated content)

Event Title: {title}
Event Description: {description}

Return ONLY a JSON object with this format:
{{"baby_friendly": true}} or {{"baby_friendly": false}}

No explanation needed, just the JSON."""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Using mini for cost efficiency
            messages=[
                {"role": "system", "content": "You are a helpful assistant that analyzes event descriptions. Always respond with valid JSON."},
                {"role": "user", "content": prompt}
            ],
            temperature=0,  # Deterministic output
            response_format={"type": "json_object"}
        )
        
        result = json.loads(response.choices[0].message.content)
        return result.get("baby_friendly", False)
    
    except Exception as e:
        print(f"Error processing event '{title}': {e}")
        return False  # Default to False on error

print("✅ Function defined successfully!")


✅ Function defined successfully!


In [5]:
# Test the function on first event
test_event = df.iloc[0]
result = extract_baby_friendly(test_event['title'], test_event['description'])

print(f"Test event: {test_event['title']}")
print(f"Baby-friendly: {result}")
print("\n✅ Test successful!")


Test event: 1.Open House New York
Baby-friendly: True

✅ Test successful!


In [6]:
# Process all events with progress bar
print("Extracting baby-friendly metadata for all events...")
print("This will take ~2-3 minutes for 100 events.\n")

baby_friendly_flags = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing events"):
    is_baby_friendly = extract_baby_friendly(row['title'], row['description'])
    baby_friendly_flags.append(is_baby_friendly)
    
    # Rate limiting: sleep briefly to avoid hitting API limits
    time.sleep(0.1)

# Add to dataframe
df['baby_friendly'] = baby_friendly_flags

print(f"\n✅ Extraction complete!")
print(f"Baby-friendly events: {sum(baby_friendly_flags)} out of {len(df)} ({sum(baby_friendly_flags)/len(df)*100:.1f}%)")


Extracting baby-friendly metadata for all events...
This will take ~2-3 minutes for 100 events.



Processing events: 100%|██████████| 99/99 [00:56<00:00,  1.75it/s]


✅ Extraction complete!
Baby-friendly events: 45 out of 99 (45.5%)


In [7]:
# Preview some baby-friendly events
baby_friendly_events = df[df['baby_friendly'] == True]
print(f"Sample baby-friendly events:")
baby_friendly_events[['title', 'baby_friendly']].head(10)


Sample baby-friendly events:


,title,baby_friendly
0,1.Open House New York,True
1,2.Tompkins Square Park Halloween Dog Parade,True
2,3.Gowanus Open Studios,True
4,5.The Amazing Maize Maze,True
5,6.Trick-or-Streets,True
6,7.Longest Table at Gotham Park,True
9,10.Dear New York at Grand Central Terminal,True
11,12.Lincoln Center's Festival of Firsts,True
14,15.Ascarium at the New York Aquarium,True
15,16.Bronx Zoo Harvest Glow,True


## 3. Create Embeddings

Generate semantic embeddings using OpenAI's `text-embedding-3-small` model.

**✅✅✅ Embedding Strategy:**
- Combine title + description for rich semantic representation
- Use OpenAI's latest embedding model (1536 dimensions)
- Event-level chunks: 1 event = 1 embedding


In [8]:
def create_embedding(text: str) -> list[float]:
    """
    Create embedding for text using OpenAI.
    
    Args:
        text: Text to embed
    
    Returns:
        list[float]: 1536-dimensional embedding vector
    """
    try:
        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=text
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"Error creating embedding: {e}")
        return [0.0] * 1536  # Return zero vector on error

print("✅ Embedding function defined!")


✅ Embedding function defined!


In [9]:
# Create text for embedding (title + description)
df['embedding_text'] = df['title'] + " " + df['description']

# Generate embeddings for all events
print("Generating embeddings for all events...")
print("This will take ~1-2 minutes for 100 events.\n")

embeddings = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Creating embeddings"):
    embedding_text = row['embedding_text']
    print(embedding_text)
    embedding = create_embedding(row['embedding_text'])
    embeddings.append(embedding)
    
    # Rate limiting
    time.sleep(0.05)

# Convert to numpy array
embeddings_array = np.array(embeddings)

print(f"\n✅ Created {len(embeddings)} embeddings")
print(f"Embedding shape: {embeddings_array.shape}")


Generating embeddings for all events...
This will take ~1-2 minutes for 100 events.



Creating embeddings:   0%|          | 0/99 [00:00<?, ?it/s]

1.Open House New York Admit it, are you a nosy New Yorker? Same here. Then you can't missOpen House New Yorkthis year. The annual event opens the doors of typically off-limit spaces during its festival running from October 17 through 19. This year, more than 300 sites are on the roster, including an exclusive rooftop garden in Rockefeller Center, a city compost facility, an Ellis Island boat and a private gallery in the World Trade Center. Even sites that are public, like Wagner Park and Astoria Pool, offer special behind-the-scenes experiences. Nearly half of the festival is fully free and open to the public—no advance registration or tickets are required. OnOpen House New York's website, you can filter to find places that fit your area of interest, such as affordable housing, building with steel and family-friendly spots. Plus, you can search for places in your borough, at certain times and that offer accessible features.


Creating embeddings:   1%|          | 1/99 [00:00<00:19,  4.99it/s]

2.Tompkins Square Park Halloween Dog Parade The Village Halloween Parade is fun and all, but does it have a plethora of puppies in adorable outfits? For that, you’ll have to head to this annual dog parade held this year on Sunday, October 19 (this year marks 35 years of this amazing event!). The getups are remarkably elaborate and conceptual—no surprise given the prizes up for grabs for Best in Show. This year's parade will feature adoptable rescue dogs from ASPCA, Social Tees, and Pup Starz who will serve as grand marshals. As a special event this year, artistJordan Sondlerwill be popping up at Romeo's (118 St Marks) offering custom pet portraits during the parade. You can book ahead using the linkhere. Festivities run from from 1-4pm. Though the exact parade route hasn't been announced, it typically runs on Avenue B from E. 6th Street to E. 14th Street, with staging on E. 7th Street between Avenue A and Avenue B. Follow along on theTompkins Square Park Dog Run's Instagram accountfor 

...

97.Swingers NoMad A bucolic 1920s English country golf club is on its way to NYC's concrete jungle! But with a twist.Swingers NoMad, a "crazy mini-golf course" and entertainment complex straight from London brought with it three nine-hole golf courses across 23,000 square feet under 20-foot-high ceilings. "Crazy golf" is a British spin on mini-golf, but it's for a 21-and-over audience since craft cocktails are served by caddies on the course. Plus, there are plenty of food options to pair with your drinks.


Creating embeddings:  98%|█████████▊| 97/99 [00:25<00:00,  4.23it/s]

98.The Brooklyn Flea (Dumbo) The Brooklyn Flea is undoubtedly one of the most popularflea marketsto hit in NYC if you're looking for the best selection of throwback wares and records, which you certainly wouldn’t find in just anyvintage clothing storeorrecord storein the city. The food selection is also top-notch since the creators also operate one of the city’s best food markets:Smorgasburg. The Brooklyn Flea DUMBO is now open for the season. Brooklyn Flea also operates in Chelsea year-round on Saturdays and Sundays.


Creating embeddings: 100%|██████████| 99/99 [00:26<00:00,  3.81it/s]


99.Shake Rattle & Roll Dueling Pianos Every Saturday night, two piano men battle it out to prove who is truly the master of all 88 keys, with a playlist decided entirely by the audience. Whether you’re in the mood for Billy Joel, Christina Aguilera or current chart toppers, these pianists are up for the challenge. But they expect you to do your part by singing along, but from home.Grab a ticket and request songs in advance.

✅ Created 99 embeddings
Embedding shape: (99, 1536)


## 4. Save Processed Data

Save the enriched data and embeddings to disk.


In [10]:
# Create processed directory if it doesn't exist
processed_dir = Path("../data/processed")
processed_dir.mkdir(parents=True, exist_ok=True)

# Save CSV (without embeddings - too large)
csv_path = processed_dir / "events_with_metadata.csv"
df_to_save = df.drop(columns=['embedding_text'])  # Remove temporary column
df_to_save.to_csv(csv_path, index=False)

# Save embeddings as numpy array
embeddings_path = processed_dir / "embeddings.npy"
np.save(embeddings_path, embeddings_array)

print(f"✅ Saved processed data to:")
print(f"  - CSV: {csv_path}")
print(f"  - Embeddings: {embeddings_path}")
print(f"\nFile sizes:")
print(f"  - CSV: {csv_path.stat().st_size / 1024:.1f} KB")
print(f"  - Embeddings: {embeddings_path.stat().st_size / 1024:.1f} KB")


✅ Saved processed data to:
  - CSV: ../data/processed/events_with_metadata.csv
  - Embeddings: ../data/processed/embeddings.npy

File sizes:
  - CSV: 83.3 KB
  - Embeddings: 1188.1 KB


## 5. Initialize Qdrant & Upload Events

Set up Qdrant vector database and upload all events with embeddings and metadata.

**✅✅✅ Vector Database Setup:**
- Using Qdrant local mode (no server needed)
- 1536-dimensional vectors (OpenAI embedding size)
- Cosine similarity for semantic search
- Metadata fields: event_id, title, description, url, baby_friendly


In [11]:
# Initialize Qdrant client (local mode)
qdrant_client = QdrantClient(path="../local_qdrant")

collection_name = "nyc_events"

# Delete collection if it exists (fresh start)
try:
    qdrant_client.delete_collection(collection_name)
    print(f"Deleted existing collection '{collection_name}'")
except:
    print(f"Collection '{collection_name}' doesn't exist yet")

# Create new collection
qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=1536,  # OpenAI text-embedding-3-small dimension
        distance=Distance.COSINE
    )
)

print(f"\n✅ Created collection '{collection_name}'")


Deleted existing collection 'nyc_events'

✅ Created collection 'nyc_events'


In [12]:
# Prepare points for upload
points = []

for idx, row in df.iterrows():
    point = PointStruct(
        id=idx,  # Use DataFrame index as ID
        vector=embeddings[idx],
        payload={
            "event_id": row['event_id'],
            "title": row['title'],
            "description": row['description'],
            "url": row['url'],
            "baby_friendly": bool(row['baby_friendly'])  # Ensure boolean type
        }
    )
    points.append(point)

# Upload to Qdrant
print(f"Uploading {len(points)} events to Qdrant...")
qdrant_client.upsert(
    collection_name=collection_name,
    points=points
)

print(f"\n✅ Successfully uploaded {len(points)} events to Qdrant!")

# Verify collection info
collection_info = qdrant_client.get_collection(collection_name)
print(f"\nCollection info:")
print(f"  - Vectors count: {collection_info.vectors_count}")
print(f"  - Points count: {collection_info.points_count}")


Uploading 99 events to Qdrant...

✅ Successfully uploaded 99 events to Qdrant!

Collection info:
  - Vectors count: None
  - Points count: 99


## 6. Test Retrieval

Test semantic search and metadata filtering.


In [13]:
def search_events(query: str, top_k: int = 5, baby_friendly_only: bool = False):
    """
    Search for events using semantic similarity.
    
    Args:
        query: Search query
        top_k: Number of results to return
        baby_friendly_only: If True, only return baby-friendly events
    
    Returns:
        List of search results
    """
    # Create embedding for query
    query_embedding = create_embedding(query)
    
    # Prepare filter if needed
    search_filter = None
    if baby_friendly_only:
        from qdrant_client.models import Filter, FieldCondition, MatchValue
        search_filter = Filter(
            must=[
                FieldCondition(
                    key="baby_friendly",
                    match=MatchValue(value=True)
                )
            ]
        )
    
    # Search Qdrant
    results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        query_filter=search_filter,
        limit=top_k
    )
    
    return results

print("✅ Search function defined!")


✅ Search function defined!


In [14]:
# Test 1: General search
query1 = "fun outdoor activity for kids"
print(f"Query: '{query1}'\n")

results1 = search_events(query1, top_k=5)

for i, result in enumerate(results1, 1):
    print(f"{i}. {result.payload['title']}")
    print(f"   Score: {result.score:.3f}")
    print(f"   Baby-friendly: {result.payload['baby_friendly']}")
    print(f"   Description: {result.payload['description'][:100]}...\n")


Query: 'fun outdoor activity for kids'

1. 69.Laser tag at mini-bowling at Area 53
   Score: 0.367
   Baby-friendly: False
   Description: Tucked away on Bridge Street in an old factory basement, this two-story playscape for kidsandadults ...

2. 16.Bronx Zoo Harvest Glow
   Score: 0.307
   Baby-friendly: True
   Description: The only thing better than the Bronx Zoo is the Bronx Zoo at night. The famed zoo's annual family-fr...

3. 15.Ascarium at the New York Aquarium
   Score: 0.303
   Baby-friendly: True
   Description: Celebrate the season with under-the-sea animals at the New York Aquarium's Ascarium. Kids can enjoy ...

4. 89.Wild Captives archery studio
   Score: 0.291
   Baby-friendly: False
   Description: Wild Captives, the nation’s first female- and LGBTQ-owned archery studio, is now open. It's a place ...

5. 7.Longest Table at Gotham Park
   Score: 0.277
   Baby-friendly: True
   Description: Clear your calendar, grab your casserole dish and prepare to dine al fresco under 

/var/folders/36/s6cm4m0959x3z451g1xvcyy40000gn/T/ipykernel_37284/339131513.py:30: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(


In [15]:
# Test 2: Baby-friendly filtered search
query2 = "art museum"
print(f"Query: '{query2}' (baby-friendly only)\n")

results2 = search_events(query2, top_k=5, baby_friendly_only=True)

for i, result in enumerate(results2, 1):
    print(f"{i}. {result.payload['title']}")
    print(f"   Score: {result.score:.3f}")
    print(f"   Baby-friendly: {result.payload['baby_friendly']}")
    print(f"   Description: {result.payload['description'][:100]}...\n")


Query: 'art museum' (baby-friendly only)

1. 43.Arte Museum
   Score: 0.568
   Baby-friendly: True
   Description: Lose yourself in immersive digital art, evocative soundscapes and custom-crafted scents at the new A...

2. 72.The newly renovated Rockefeller Wing at The Met
   Score: 0.432
   Baby-friendly: True
   Description: After a four-year renovation, The Metropolitan Museum of Art has reopened its galleries dedicated to...

3. 65."Breaking the Mold: Brooklyn Museum at 200" exhibit
   Score: 0.410
   Baby-friendly: True
   Description: The Brooklyn Museum is celebrating a big birthday. As the museum turns 200, it’s marking the occasio...

4. 25."Monet and Venice" at Brooklyn Museum
   Score: 0.383
   Baby-friendly: True
   Description: For those who have long romanticized the floating city of Venice, theBrooklyn Museum's new exhibit w...

5. 56.“Foot Fountain (pink)”
   Score: 0.368
   Baby-friendly: True
   Description: There’s a sculpture on the High Line right now that’s causin

/var/folders/36/s6cm4m0959x3z451g1xvcyy40000gn/T/ipykernel_37284/339131513.py:30: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(


In [16]:
# Test 3: Semantic search (mood/vibe)
query3 = "romantic date night"
print(f"Query: '{query3}'\n")
print("✅✅✅ Note: No explicit mood tags needed! Semantic search captures vibe naturally.\n")

results3 = search_events(query3, top_k=5)

for i, result in enumerate(results3, 1):
    print(f"{i}. {result.payload['title']}")
    print(f"   Score: {result.score:.3f}")
    print(f"   Baby-friendly: {result.payload['baby_friendly']}")
    print(f"   Description: {result.payload['description'][:100]}...\n")


Query: 'romantic date night'

✅✅✅ Note: No explicit mood tags needed! Semantic search captures vibe naturally.

1. 41.Latin Mix Saturdays
   Score: 0.322
   Baby-friendly: False
   Description: Experience the ultimate Saturday night at Time Out Market’s Rooftop Latin Mix Party! Join resident D...

2. 57.Rooftop Cinema Club
   Score: 0.316
   Baby-friendly: False
   Description: Rooftop Cinema Club takes movie-going to a whole new level—literally. This rooftop film series at a ...

3. 34.Handmade Happy Hour
   Score: 0.314
   Baby-friendly: False
   Description: Get your hands dirty and jump into the world of ceramics with Handmade Happy Hour—Greenwich House Po...

4. 79.Friday Night Vibes at Time Out Market
   Score: 0.311
   Baby-friendly: False
   Description: Start your weekend off right at Time Out Market New York’s stunning rooftop! Friday Night Vibes gets...

5. 88.Act & Sip NYC
   Score: 0.308
   Baby-friendly: False
   Description: If you're not a paint-and-sip kind of person, 

/var/folders/36/s6cm4m0959x3z451g1xvcyy40000gn/T/ipykernel_37284/339131513.py:30: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(


In [18]:
# Create backend directory if it doesn't exist
backend_dir = Path("../backend")
backend_dir.mkdir(parents=True, exist_ok=True)

print("Creating vector_store.py utility module...")

vector_store_code = '''"""Vector store operations for NYC Event Recommender.

This module handles all interactions with the Qdrant vector database,
including initialization, uploading events, and searching.
"""

from typing import List, Optional, Dict, Any
from qdrant_client import QdrantClient
from qdrant_client.models import (
    Distance,
    VectorParams,
    PointStruct,
    Filter,
    FieldCondition,
    MatchValue
)
from openai import OpenAI
import os


class VectorStore:
    """Manages vector database operations for event search."""
    
    def __init__(self, path: str = "./local_qdrant", collection_name: str = "nyc_events"):
        """Initialize vector store.
        
        Args:
            path: Path to local Qdrant storage
            collection_name: Name of the collection to use
        """
        self.client = QdrantClient(path=path)
        self.collection_name = collection_name
        self.openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    
    def initialize_collection(self, vector_size: int = 1536) -> None:
        """Create or recreate the collection.
        
        Args:
            vector_size: Dimension of embedding vectors
        """
        # Delete if exists
        try:
            self.client.delete_collection(self.collection_name)
        except:
            pass
        
        # Create new collection
        self.client.create_collection(
            collection_name=self.collection_name,
            vectors_config=VectorParams(
                size=vector_size,
                distance=Distance.COSINE
            )
        )
    
    def upload_events(self, events: List[Dict[str, Any]], embeddings: List[List[float]]) -> None:
        """Upload events with embeddings to vector store.
        
        Args:
            events: List of event dictionaries with metadata
            embeddings: List of embedding vectors
        """
        points = []
        for idx, (event, embedding) in enumerate(zip(events, embeddings)):
            point = PointStruct(
                id=idx,
                vector=embedding,
                payload=event
            )
            points.append(point)
        
        self.client.upsert(
            collection_name=self.collection_name,
            points=points
        )
    
    def create_embedding(self, text: str) -> List[float]:
        """Create embedding for text using OpenAI.
        
        Args:
            text: Text to embed
        
        Returns:
            List of floats representing the embedding vector
        """
        response = self.openai_client.embeddings.create(
            model="text-embedding-3-small",
            input=text
        )
        return response.data[0].embedding
    
    def search_events(
        self,
        query: str,
        top_k: int = 10,
        filters: Optional[Dict[str, Any]] = None
    ) -> List[Dict[str, Any]]:
        """Search for events using semantic similarity.
        
        Args:
            query: Search query string
            top_k: Number of results to return
            filters: Optional metadata filters, e.g. {"baby_friendly": True}
        
        Returns:
            List of search results with scores and payloads
        """
        # Create query embedding
        query_embedding = self.create_embedding(query)
        
        # Build filter if provided
        search_filter = None
        if filters:
            conditions = []
            for key, value in filters.items():
                conditions.append(
                    FieldCondition(
                        key=key,
                        match=MatchValue(value=value)
                    )
                )
            if conditions:
                search_filter = Filter(must=conditions)
        
        # Search
        results = self.client.search(
            collection_name=self.collection_name,
            query_vector=query_embedding,
            query_filter=search_filter,
            limit=top_k
        )
        
        # Format results
        formatted_results = []
        for result in results:
            formatted_results.append({
                "score": result.score,
                "event": result.payload
            })
        
        return formatted_results
    
    def get_collection_info(self) -> Dict[str, Any]:
        """Get information about the collection.
        
        Returns:
            Dictionary with collection statistics
        """
        info = self.client.get_collection(self.collection_name)
        return {
            "vectors_count": info.vectors_count,
            "points_count": info.points_count,
            "status": info.status
        }
'''

# Write to file
vector_store_path = backend_dir / "vector_store.py"
vector_store_path.write_text(vector_store_code)

print(f"✅ Created {vector_store_path}")
print(f"\nModule includes:")
print("  - VectorStore class")
print("  - initialize_collection()")
print("  - upload_events()")
print("  - search_events()")
print("  - create_embedding()")
print("  - get_collection_info()")


Creating vector_store.py utility module...
✅ Created ../backend/vector_store.py

Module includes:
  - VectorStore class
  - initialize_collection()
  - upload_events()
  - search_events()
  - create_embedding()
  - get_collection_info()


## Summary & Next Steps

### ✅ Completed Tasks:

1. **Loaded raw data** from Notebook 1
2. **Extracted baby-friendly metadata** using GPT-4 Mini
   - Simple boolean flag: suitable for infants/toddlers
   - No mood or energy tags needed!
3. **Generated embeddings** using OpenAI text-embedding-3-small
   - 1536-dimensional vectors
   - Combined title + description for rich semantics
4. **Saved processed data**
   - CSV: `data/processed/events_with_metadata.csv`
   - Embeddings: `data/processed/embeddings.npy`
5. **Initialized Qdrant** vector database
   - Local mode (no server needed)
   - Cosine similarity search
6. **Uploaded all events** with metadata
7. **Tested retrieval** with multiple queries
   - General semantic search ✅
   - Baby-friendly filtering ✅
   - Mood/vibe queries ✅
8. **Created utility module** (`backend/vector_store.py`)

### 📊 Results:

- **Events processed:** Check cell output above
- **Baby-friendly events:** Check percentage above
- **Vector database:** Operational and ready
- **Search quality:** Semantic search captures nuance without explicit tags!

### 🎯 Next Steps:

**Move to Notebook 3:** Agentic RAG Pipeline
- Build 2-agent system (Retrieval + Response)
- Create FastAPI backend
- Implement LangSmith logging
- Test end-to-end queries

---

**✅✅✅ Key Insight:** By keeping metadata extraction simple (just baby-friendly), we saved significant time while still enabling powerful semantic search. The embeddings naturally capture mood, vibe, and context without explicit tagging!
